In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import pandas as pd 
import datetime
import numpy as np 
import joblib
import re  
import jieba 
import json
import time
from openai import AzureOpenAI
from pandasql import sqldf
from preprocessing import extract_versions, extract_models, WordCut

from datasets import Dataset 
from datasets import Features, ClassLabel, Sequence, Value
from datasets import load_from_disk, load_dataset

In [3]:
# pandasql查询函数需要的环境
pysqldf = lambda q: sqldf(q, globals())

In [4]:
def extract_keywords(question, all_model_list, wc):
    original_question = question
    model_list, question = extract_models(question, all_model_list)
    version_list, question = extract_versions(question)
    key_words = wc.cut(question)
    key_words = [i for i in key_words if ((i.find("model")<0) and (i.find("version")<0))]
    return {"question": original_question, "model": model_list, "version": version_list, "keywords": key_words}

In [25]:
# 原始数据处理
def format_model(x):
    model_list = x.split(',')
    model_list = [i.strip().lower().replace(" ", "") for i in model_list]
    new_list = [model_list[0]]
    i = 1
    while i < len(model_list):
        if (i != len(model_list) - 1) and (model_list[i-1] == model_list[i]):
            new_list.append(model_list[i]+model_list[i+1])
            if i < len(model_list) - 1:
                i += 2
            else:
                break
        elif (i != len(model_list) - 1) and (model_list[i-1] != model_list[i]):
            new_list.append(model_list[i])
            i += 1
        elif (model_list[i] == "上下水") or (model_list[i] == "air"):
            for j in range(len(new_list)):
                if model_list[i-1] == new_list[j]:
                    new_list.pop(j)
                    break
            new_list.append(model_list[i-1]+model_list[i])
            i += 1
        else:
            new_list.append(model_list[i])
            break
    return new_list

# 原始数据处理
def format_model2(x):
    model_list = x.split(',')
    model_list = [i.strip().lower() for i in model_list]
    new_list = [model_list[0]]
    i = 1
    while i < len(model_list):
        if (i != len(model_list) - 1) and (model_list[i-1] == model_list[i]):
            new_list.append(model_list[i]+model_list[i+1])
            if i < len(model_list) - 1:
                i += 2
            else:
                break
        elif (model_list[i][:3] == "上下水") or (model_list[i][:3] == "air") or (model_list[i][:3] == "pro") or (model_list[i][:4] == "pure"):
            for j in range(len(new_list)-1, -1, -1):
                if model_list[i-1] == new_list[j]:
                    new_list.pop(j)
                    break
            new_list.append(model_list[i-1]+model_list[i])
            i += 1
        elif (i != len(model_list) - 1) and (model_list[i-1] != model_list[i]):
            new_list.append(model_list[i])
            i += 1
        else:
            new_list.append(model_list[i])
            break
    return new_list

def format_all_models(x, dim_df):
    new_list = []
    for i in x:
        if i.find("全型号") >= 0:
            end_idx = i.find("全型号")
            name = i[:end_idx]
            new_list += [j for j in dim_df[dim_df['cat_name'] == name].model.tolist() if j not in x]
        else:
            new_list.append(i)
    return new_list

def format_series(x, dim_df):
    def contains_chinese(s):
        return re.search('[\u4e00-\u9fff]', s) is not None
    new_list = []
    for i in x:
        if i.find("系列") >= 0:
            end_idx = i.find("系列")
            name = i[:end_idx]
            new_list += [j for j in dim_df[(dim_df.model.str.find(name)>=0) & (
                dim_df.model.apply(lambda x: not contains_chinese(x)))].model.tolist() if j not in x]
            new_list += [i]
        else:
            new_list.append(i)
    return new_list

In [26]:
def count_gt(x):
    if str(x) == "nan":
        return 0
    else:
        return len(x.split(","))  
    
def find_non_chinese_substrings(s):
    # 正则表达式解释：
    # [^\u4e00-\u9fff\W]+ 匹配非中文字符和非ASCII标点的连续字符
    # 但这样会排除空格，所以我们需要允许空格存在
    # 我们使用(?:[^\u4e00-\u9fff\W]| )+ 来实现这一点，(?:) 是非捕获组，用于匹配模式但不作为捕获结果返回
    # [^\u4e00-\u9fff\W] 匹配非中文且非标点的字符，| 表示或，空格 ' ' 被显式允许
    pattern = r'(?:[^\u4e00-\u9fff\W]| )+'
    
    # 使用findall方法查找所有匹配项
    matches = re.findall(pattern, s)
    
    # 过滤掉只包含空格的字符串
    matches = [match for match in matches if not match.isspace()]
    
    return matches

def clean_string(s):
    s = s.replace(" ", "").lower()
    return s

def find_model(x, all_model_list):
    x = x.replace("\n", "") 
    x = find_non_chinese_substrings(x)
    result = [clean_string(s) for s in x]
    return [model for model in all_model_list if model in result]

def find_cat(x, all_cat_list):
    return [name for name in all_cat_list if name in x]   

def filter_model(x, model_list):
    x = x.split(",")
    for model in model_list:
        if model in x:
            return True
    return False

def find_error_with_reason(a):
    # 第一次匹配“错误xxx”
    pattern1 = r"错误\s*\d+"
    matches1 = re.findall(pattern1, a)
    
    # 第二次匹配“错误原因xxx”
    pattern2 = r"错误原因\s*\d+"
    matches2 = re.findall(pattern2, a)

    # 合并两次匹配的结果
    matches = matches1 + matches2
    
    return [name.replace(" ", "").replace("原因", "") for name in matches]

def filter_reason(x, query_reason_list):
    reason_list = find_error_with_reason(x)
    for name in query_reason_list:
        if name in reason_list:
            return True 
    return False

def transform_model_name(x, all_model_list):
    x = x.replace("\n", "") 
    candidates = find_non_chinese_substrings(x)
    for name in candidates:
        cleaned_name = clean_string(name)
        for model in all_model_list:
            if cleaned_name == model:
                x = x.replace(name, model)
                break
    return x 

def remove_model_name(x, all_model_list):
    x = x.replace("\n", "") 
    candidates = find_non_chinese_substrings(x)
    for name in candidates:
        if clean_string(name) in all_model_list:
            x = x.replace(name, "")
    return x 

In [28]:
def find_non_chinese_substrings_v2(s):
    pattern = r'(?:[^\u4e00-\u9fff\W]| )+(?:上下水(?:版(?:本)?)?)?'
    matches = re.finditer(pattern, s)

    substrings_with_positions = []
    for match in matches:
        start, end = match.span()
        substring = match.group()

        # 去除左右两边的空格并调整位置
        stripped_substring = substring.strip()
        start += len(substring) - len(substring.lstrip())
        end -= len(substring) - len(substring.rstrip())

        if stripped_substring and not stripped_substring.isspace():
            substrings_with_positions.append((stripped_substring, start, end))

    return substrings_with_positions


def clean_string(s):
    s = s.replace(" ", "").lower()
    return s

def find_model_v2(x, all_model_dict):
    substrings_with_positions = find_non_chinese_substrings_v2(x)
    results = []

    for substring, start, end in substrings_with_positions:
        cleaned_substring = clean_string(substring)
        cleaned_substring = cleaned_substring.replace("版本", "").replace("版", "")
        if cleaned_substring in all_model_dict:
            results.append((cleaned_substring, start, end))

    return results

In [27]:
def bio_tagging(texts, labels):
    bio_words = []
    bio_tags = []
    texts = texts.lower()
    for i, char in enumerate(texts):
        tag = "O"  # 默认为 Outside

        for entity_type, spans in labels.items():
            for span in spans.values():
                for ind in span:
                    if i == int(ind[0]):
                        tag = "B-" + entity_type
                        break
                    elif int(ind[0]) < i <= int(ind[1]):
                        tag = "I-" + entity_type
                        break

        bio_words.append(char)
        bio_tags.append(tag)

    return bio_words, bio_tags

In [8]:
# 定义NER数据集

In [2]:
df2 = pd.read_csv("/data/dataset/kefu/database_with_emb20240315.csv")

In [6]:
oot = pd.read_excel("/data/dataset/kefu/国内客服助手（生产环境）_中转栈.xlsx")
oot = oot.rename(columns={"编号": "qa_id",
 "问题": "question",
 "回复1": "answer1_all",
 "回复1标题": "answer1",
 "回复2": "answer2_all",
 "回复2标题": "answer2",
 "是否解决": "if_solved",
 "提问者": "requester",
 "提问者所在组别": "requester_group",
 "提问日期": "request_time",
 "类型": "data_type", 
 "正确回复": "gt_answer"})
oot = oot.drop(["回复1附件", "回复2附件", "提问日期(供统计用)"], axis=1)

In [7]:
# oot = oot[oot.if_solved.notnull()]
# oot = oot[oot.answer1_all.notnull()]
oot["if_solved"] = oot["if_solved"].map({"已解决": 1, "未解决": 0})
oot.loc[oot.qa_id=="ICASK202308010583", "gt_answer"] = "ICWIKI202307243975"
oot.loc[oot.qa_id=="ICASK202308010582", "gt_answer"] = "ICWIKI202308210081"
# oot = oot[oot['gt_answer'].str.find("ICW")>=0]
oot = oot.rename(columns={"gt_answer": "gt_qa_id"})
oot = oot[oot.question.notnull()]

temp = oot.copy()
temp["gt_qa_id"] = temp["gt_qa_id"].astype(str).apply(lambda x: x.split(','))
temp_exploded = temp.explode("gt_qa_id")
temp_right = df2[['qa_id', 
                               'question', 
                               'answer', 
                               'model', 
                               'qa_type', 
                               'model_list', 
                               'cat_name']].copy()
query = f"""
select 
    a.*
    ,b.question as question_kg
    ,b.answer as answer_kg
    ,b.model as model
    ,b.qa_type
    ,b.model_list
    ,b.cat_name
from 
    temp_exploded a 
left join 
    temp_right b
on 
    a.gt_qa_id = b.qa_id
"""

# 使用pysqldf执行SQL查询
temp_exploded = pysqldf(query)

In [8]:
query = f"""
select 
    qa_id
    ,group_concat(question_kg) as question_kg
    ,group_concat(answer_kg) as answer_kg
    ,group_concat(model) as model
    ,group_concat(qa_type) as qa_type
    ,group_concat(model_list) as model_list
    ,group_concat(cat_name) as cat_name
from 
    temp_exploded
group by 
    qa_id
"""

# 使用pysqldf执行SQL查询
temp_exploded = pysqldf(query)

In [9]:
query = f"""
select 
    a.*
    ,b.question_kg
    ,b.answer_kg
    ,b.model
    ,b.qa_type
    ,b.model_list
    ,b.cat_name
from 
    oot a 
left join 
    temp_exploded b
on
    a.qa_id = b.qa_id
"""

# 使用pysqldf执行SQL查询
oot = pysqldf(query)

oot['gt_num'] = oot['gt_qa_id'].astype(str).apply(lambda x: count_gt(x))

In [12]:
joblib.dump(oot[["qa_id", 
     "question",	
     "gt_qa_id",
     "gt_num",
     "question_kg",
     "answer_kg",
     "model",
     "qa_type",
     "model_list",
     "cat_name",
     ]], "/data/dataset/kefu/oot20240422.json")

['/data/dataset/kefu/oot20240422.json']

In [10]:
oot = joblib.load("/data/dataset/kefu/oot20240422.json")

In [45]:
dim_df = pd.read_csv("/data/dataset/kefu/dim_df20240315.csv")
all_model_list = dim_df.model.tolist()
all_cat_list = dim_df.cat_name.unique().tolist()

In [46]:
# wc = WordCut()

In [47]:
# oot["keywords"] = oot["question"].apply(
#     lambda x: extract_keywords(x, all_model_list, wc))

In [48]:
# oot[oot.version_keywords.apply(lambda x: len(x)>0)][["question", "keywords"]] 

In [49]:
# 定义知识库

In [10]:
df_sweeping = pd.read_excel("/data/dataset/kefu/产品知识整理资料.xlsx", sheet_name="扫地机")

In [11]:
df_sweeping.loc[df_sweeping["上市时间"]==45323, "上市时间"] = datetime.datetime(2024, 2, 1, 0, 0)

In [12]:
df_mopping = pd.read_excel("/data/dataset/kefu/产品知识整理资料.xlsx", sheet_name="洗地机")

In [13]:
df_washing = pd.read_excel("/data/dataset/kefu/产品知识整理资料.xlsx", sheet_name="洗衣机")
df_washing = df_washing.iloc[:3]

In [14]:
for col in ["商品编码", "平台ID", "商品id"]:
    df_sweeping[col] = df_sweeping[col].astype(str)
    df_sweeping.loc[df_sweeping[col]=='nan', col] = np.nan
    df_mopping[col] = df_mopping[col].astype(str)
    df_mopping.loc[df_mopping[col]=='nan', col] = np.nan
    df_washing[col] = df_washing[col].astype(np.int64).astype(str)
    df_washing.loc[df_washing[col]=='nan', col] = np.nan

In [15]:
print(df_washing.shape)
print(df_mopping.shape)
print(df_sweeping.shape)

(3, 73)
(7, 87)
(14, 91)


In [16]:
all_model_dict = {}
for i in range(df_sweeping.shape[0]):
    model_name = format_model(df_sweeping.iloc[i].商品型号)[0]
    all_model_dict[model_name] = "扫地机"
    all_model_dict[model_name+"上下水版"] = "扫地机"
    all_model_dict[model_name+"上下水"] = "扫地机"
for i in range(df_mopping.shape[0]):
    model_name = format_model(df_mopping.iloc[i].商品型号)[0]
    all_model_dict[model_name] = "洗地机"
for i in range(df_washing.shape[0]):
    model_name = format_model(df_washing.iloc[i].商品型号)[0]
    all_model_dict[model_name] = "洗衣机"

In [17]:
dim_df = pd.read_csv("/data/dataset/kefu/dim_df20240315.csv")
dim_df.model = dim_df.model.apply(lambda x: x.replace("版本", "").replace("版", ""))
all_model_list = dim_df.model.tolist()
all_cat_list = dim_df.cat_name.unique().tolist()

In [18]:
for i in all_model_list:
    if i not in all_model_dict:
        print(i)
        assert dim_df.loc[dim_df["model"]==i, "cat_name"].shape[0] == 1
        all_model_dict[i] = dim_df.loc[dim_df["model"]==i, "cat_name"].iloc[0]

h6
h7
u10
洗地机全型号
智能烘干套件
智能集尘充电座
自动集尘充电座
p5
s5
t6
t7
t7pro
t7s
t7splus
g10
g10plus
t8
t8plus
米家扫地机
智能上下水及烘干套件（g10s、pro、auto的上下水）
智能上下水套件（g10spure的上下水）
g10s系列


In [19]:
all_model_dict

{'g20': '扫地机',
 'g20上下水版': '扫地机',
 'g20上下水': '扫地机',
 'p10s': '扫地机',
 'p10s上下水版': '扫地机',
 'p10s上下水': '扫地机',
 'p10spro': '扫地机',
 'p10spro上下水版': '扫地机',
 'p10spro上下水': '扫地机',
 'g10spure': '扫地机',
 'g10spure上下水版': '扫地机',
 'g10spure上下水': '扫地机',
 'g10spro': '扫地机',
 'g10spro上下水版': '扫地机',
 'g10spro上下水': '扫地机',
 'p10': '扫地机',
 'p10上下水版': '扫地机',
 'p10上下水': '扫地机',
 'g10s': '扫地机',
 'g10s上下水版': '扫地机',
 'g10s上下水': '扫地机',
 'p10pro': '扫地机',
 'p10pro上下水版': '扫地机',
 'p10pro上下水': '扫地机',
 'g10sauto': '扫地机',
 'g10sauto上下水版': '扫地机',
 'g10sauto上下水': '扫地机',
 'a10ultra': '洗地机',
 'a10ultrae': '洗地机',
 'a10': '洗地机',
 'a10plus': '洗地机',
 'a20': '洗地机',
 'a20pro': '洗地机',
 'a20air': '洗地机',
 'h1': '洗衣机',
 'h1neo': '洗衣机',
 'm1': '洗衣机',
 'h6': '吸尘器',
 'h7': '吸尘器',
 'u10': '洗地机',
 '洗地机全型号': '洗地机',
 '智能烘干套件': '扫地机',
 '智能集尘充电座': '扫地机',
 '自动集尘充电座': '扫地机',
 'p5': '扫地机',
 's5': '扫地机',
 't6': '扫地机',
 't7': '扫地机',
 't7pro': '扫地机',
 't7s': '扫地机',
 't7splus': '扫地机',
 'g10': '扫地机',
 'g10plus': '扫地机',
 't8': '扫地机',
 't8plus': '扫地机',
 '

In [20]:
filter =  {}  

In [21]:
for col in df_washing.columns:
    if col in filter:
        filter[col] = filter[col] | set(["washing"])
    else:
        filter[col] = set(["washing"])
for col in df_mopping.columns:
    if col in filter:
        filter[col] = filter[col] | set(["mopping"])
    else:
        filter[col] = set(["mopping"])
for col in df_sweeping.columns:
    if col in filter:
        filter[col] = filter[col] | set(["sweeping"])
    else:
        filter[col] = set(["sweeping"])

In [54]:
filter

{'商品编码': {'mopping', 'sweeping', 'washing'},
 '平台ID': {'mopping', 'sweeping', 'washing'},
 '商品id': {'mopping', 'sweeping', 'washing'},
 '商品型号': {'mopping', 'sweeping', 'washing'},
 '商品名字': {'mopping', 'sweeping', 'washing'},
 '商品分类': {'mopping', 'sweeping', 'washing'},
 '商品链接': {'mopping', 'sweeping', 'washing'},
 '平台': {'mopping', 'sweeping', 'washing'},
 '店铺名称': {'mopping', 'sweeping', 'washing'},
 '服务别名': {'mopping', 'sweeping', 'washing'},
 '产地': {'washing'},
 '使用方式': {'washing'},
 '颜色分类': {'sweeping', 'washing'},
 '能效等级': {'washing'},
 '排水方式': {'washing'},
 '控制方式': {'washing'},
 '开合方式': {'washing'},
 '产品类型': {'washing'},
 '电机类型': {'washing'},
 '显示屏类型': {'washing'},
 '毛重': {'washing'},
 '机器尺寸': {'washing'},
 '净重': {'washing'},
 '上市时间': {'mopping', 'sweeping', 'washing'},
 '烘干方式': {'sweeping', 'washing'},
 '烘干公斤量': {'washing'},
 '洗涤公斤量': {'washing'},
 '包装尺寸': {'mopping', 'sweeping', 'washing'},
 '烘干功率': {'washing'},
 '建议摆放尺寸': {'washing'},
 '洗衣液容量': {'washing'},
 '柔顺剂容量': {'washing'

In [55]:
# df_washing.to_csv("/data/dataset/kefu/washing.csv", index=None)
# df_mopping.to_csv("/data/dataset/kefu/mopping.csv", index=None)
# df_sweeping.to_csv("/data/dataset/kefu/sweeping.csv", index=None)

In [22]:
df_all = pd.concat([df_washing, df_mopping, df_sweeping], axis=0).reset_index(drop=True)

In [32]:
df_all["商品型号"] = df_all["商品型号"].apply(lambda x: find_model_v2(x, all_model_dict)[0][0])

In [33]:
# df_all.to_csv("/data/dataset/kefu/model_params20240620.csv", index=None)

In [9]:
df_all = pd.read_csv("/data/dataset/kefu/model_params20240620.csv")

In [22]:
# 使用csv2qa.py从模板收集数据表格问答数据

In [151]:
def read_json(json_file='/data/dataset/kefu/gen_different_keywords_different_models.jsonl'):
    df = []
    # 打开文件进行读取
    with open(json_file, 'r') as file:
        # 逐行读取
        for line in file:
            # 将每行的内容从JSON字符串转换为Python字典
            data = json.loads(line.strip())
            # 现在可以处理这个字典了
            df.append(data)
    return df 

def extract_json(x):
    result = {"template": x["template"]}
    result["replace"] = x["gen"]["replace"]
    return json.dumps(result, ensure_ascii=False)

file_list = ['/data/dataset/kefu/gen_same_keywords_for_models_v2.jsonl',
             '/data/dataset/kefu/gen_different_keywords_different_models_v2.jsonl']
df = []
for file in file_list:
    temp = pd.DataFrame(read_json(file))
    temp['type'] = file.split('/')[-1].split(".")[0]
    df.append(temp)
df = pd.concat(df, axis=0).reset_index(drop=True)

df["final_prompt"] = df.apply(lambda x: extract_json(x), axis=1)

In [152]:
df[df.cat_num==2].head()

,cat,gen,template,cat_num,type,final_prompt
6,"mopping,sweeping","{'question': 'g20和p10pro的服务别名是什么？', 'prompt': ...",[问询词0]和[问询词1]的[关键词0]是什么？,2,gen_same_keywords_for_models_v2,"{""template"": ""[问询词0]和[问询词1]的[关键词0]是什么？"", ""repl..."
7,"mopping,sweeping",{'question': 'p10pro标准版和g10sauto标准版的清水箱续航和额定输入...,[问询词0]和[问询词1]的[关键词0]和[关键词1]是什么？,2,gen_same_keywords_for_models_v2,"{""template"": ""[问询词0]和[问询词1]的[关键词0]和[关键词1]是什么？""..."
8,"mopping,sweeping",{'question': 'g10sauto和g20上下水版的清扫模式和烘干时长和标配是什么...,[问询词0]和[问询词1]的[关键词0]和[关键词1]和[关键词2]是什么？,2,gen_same_keywords_for_models_v2,"{""template"": ""[问询词0]和[问询词1]的[关键词0]和[关键词1]和[关键词..."
9,"mopping,sweeping",{'question': 'a20pro和p10上下水版和a10plus的自清洁模式选择是什...,[问询词0]和[问询词1]和[问询词2]的[关键词0]是什么？,2,gen_same_keywords_for_models_v2,"{""template"": ""[问询词0]和[问询词1]和[问询词2]的[关键词0]是什么？""..."
10,"mopping,sweeping",{'question': 'a10plus和g10sauto标准版和a10ultrae的助力...,[问询词0]和[问询词1]和[问询词2]的[关键词0]和[关键词1]是什么？,2,gen_same_keywords_for_models_v2,"{""template"": ""[问询词0]和[问询词1]和[问询词2]的[关键词0]和[关键词..."


In [153]:
df["primary_key_num"] = df["template"].apply(lambda x: x.count("问询"))

In [154]:
df.groupby(["type", "cat_num", "primary_key_num"]).count()

cat  gen  \
type                                       cat_num primary_key_num             
gen_different_keywords_different_models_v2 1       4                  3    3   
                                                   6                  3    3   
                                           2       4                  3    3   
                                                   6                  3    3   
                                           3       4                  3    3   
                                                   6                  3    3   
gen_same_keywords_for_models_v2            1       2                  3    3   
                                                   3                  3    3   
                                           2       2                  3    3   
                                                   3                  3    3   
                                           3       2                  3    3   
                                                   3                  3    3   

                                                                    template  \
type                                       cat_num primary_key_num             
gen_different_keywords_different_models_v2 1       4                       3   
                                                   6                       3   
                                           2       4                       3   
                                                   6                       3   
                                           3       4                       3   
                                                   6                       3   
gen_same_keywords_for_models_v2            1       2                       3   
                                                   3                       3   
                                           2       2                       3   
                                                   3                       3   
                                           3       2                       3   
                                                   3                       3   

                                                                    final_prompt  
type                                       cat_num primary_key_num                
gen_different_keywords_different_models_v2 1       4                           3  
                                                   6                           3  
                                           2       4                           3  
                                                   6                           3  
                                           3       4                           3  
                                                   6                           3  
gen_same_keywords_for_models_v2            1       2                           3  
                                                   3                           3  
                                           2       2                           3  
                                                   3                           3  
                                           3       2                           3  
                                                   3                           3

In [27]:
# 模板收集数据的转义表达和扩增

In [673]:
prompt = """
输入json含义如下：
1. template是一个问句，问句里面用[]符号括起来的部分是命名实体，其中“问询词”开头的是家庭扫拖机器人或者洗衣机
2. replace是上述每个命名实体的具体值

请你按以下步骤操作：
1. 将template里面的每个命名实体替换成replace里面对应的值，当要替换“关键词”开头的所有命名实体的时候先把/()等特殊符号先去除再替换
2. 再替换后的句子中去除标记实体的[]符号，然后把整个句子改写得语法通顺但是语义不能改变，改写中请保持“问询词”开头的命名实体作为整体不变
3. 检查改写后的句子，如果句子不通顺则重新改写使句子通顺
4. 在新生成的句子中，提取与原始“关键词”开头的所有命名实体对应的关键词并用括号[]括起来，每个关键词在语义上应该能代表它对应的原始实体的语义，所有关键词的数量要与原始“关键词”开头的命名实体数量一致

请理解以下几个例子：
例子1：
输入：
{{"template": "[问询词0]的[关键词0]是多少？", "replace": {{"[问询词0]": "g20", "[关键词0]": "是否支持上下水"}}}}
输出：
{{"sentence": "请问g20是否[有上下水功能]？", "replace": {{"是否支持上下水": "有上下水功能"}}}}

例子2：
输入：
{{"template": "[问询词0-0]和[问询词0-1]的[关键词0-0]是什么、[问询词1-0]的[关键词1-0]和[关键词1-1]是多少？", "replace": {{"[问询词0-0]": "g20", "[问询词0-1]": "a10", "[关键词0-0]": "是否支持银离子抑菌", "[问询词1-0]": "a10", "[关键词1-0]": "是否带遥控器", "[关键词1-1]": "水箱容量"}}}}
输出：
{{"sentence": "请问g20和a10是否[可以对银离子抑菌]，另外a10[带遥控器]吗、它[水箱有多大]？", "replace": {{"是否支持银离子抑菌": "可以对银离子抑菌", "是否带遥控器": "带遥控器", "水箱容量": "水箱有多大"}}}}

例子3：
输入：
{{"template": "[问询词0]的[关键词0]是多少？", "replace": {{"[问询词0]": "g20", "[关键词0]": "浸泡功能"}}}}
绝对避免的输出(提取的实体包含不必要的东西，不简洁)包括：
{{"sentence": "请问g20[有哪些浸泡功能]？", "replace": {{"浸泡功能": "有哪些浸泡功能"}}}}
{{"sentence": "g20[浸泡功能都有什么]？", "replace": {{"浸泡功能": "浸泡功能都有什么"}}}}
希望的输出：
{{"sentence": "请问g20有何[浸泡功能]？", "replace": {{"浸泡功能": "浸泡功能"}}}}


参考上面的例子，请对每个输入给出{}种不同的说法, 每种说法都要按照上述4个步骤依次做一遍，每种说法对应一个json，放在列表里面返回, 除此之外不要给出任务其他的信息，即如果要求给出n种说法，则输出
[json_0, json_1, json_2, ..., json_n-1]，如果要给出1种说法，则输出[json_0]

输入：
{}
输出：
"""

In [329]:
client = AzureOpenAI(
    api_version="2024-02-15-preview",
    azure_endpoint="https://csagent.openai.azure.com/",
    api_key="346ac6661e314a9d8b91b6a99202ba42"
)

In [588]:
def generate_answer(prompt, input_json, n, model="gpt-4-8k"): # model = "deployment_name"
    response = client.chat.completions.create(
    model=model, # model = "deployment_name".
    messages=[
        {"role": "user", "content": prompt.format(n, input_json)},
    ]
    )
    return response.choices[0].message.content

In [37]:
result = []
prev_time = time.time()
for i in range(df.shape[0]):
    if i % 10 == 0:
        print(i)
        cur_time = time.time()
        print(cur_time-prev_time)
        prev_time = cur_time
    if df["cat_num"].iloc[i] == 1:
        n = 12
    else:
        n = 4
    item = generate_answer(prompt, df["final_prompt"].iloc[i], n)
    result.append(item)
    joblib.dump(item, "/data/dataset/kefu/gpt4_template_v2/{}.jsonl".format(i))

0
0.0004031658172607422
10
535.0191197395325
20
438.8204011917114
30
921.7273585796356


In [155]:
df.shape

(36, 7)

In [156]:
result = []
for i in range(df.shape[0]):
    item = joblib.load("/data/dataset/kefu/gpt4_template_v2/{}.jsonl".format(i))
    result.append(item)

In [157]:
len(result)

36

In [158]:
error_list = []
for i in range(df.shape[0]):
    if (df.iloc[i]["type"]=="gen_different_keywords_different_models_v2"
            )&(df.iloc[i]["cat_num"]==1)|(df.iloc[i]["type"]=="gen_different_keywords_different_models_v2"
            )&(df.iloc[i]["cat_num"]==2):
        error_list.append(i)    

In [678]:
result = []
prev_time = time.time()
for index, i in enumerate(error_list):
    if index % 10 == 0:
        print(index)
        cur_time = time.time()
        print(cur_time-prev_time)
        prev_time = cur_time
    if df["cat_num"].iloc[i] == 1:
        n = 12
    else:
        n = 4
    item = generate_answer(prompt, df["final_prompt"].iloc[i], n)
    result.append(item)
    joblib.dump(item, "/data/dataset/kefu/gpt4_template_v2_2/{}.jsonl".format(i))

0
0.000255584716796875
10
615.4492988586426


In [162]:
result = []
for i in range(df.shape[0]):
    if i not in error_list:
        item = joblib.load("/data/dataset/kefu/gpt4_template_v2/{}.jsonl".format(i))
    else:
        item = joblib.load("/data/dataset/kefu/gpt4_template_v2_2/{}.jsonl".format(i))
    result.append(item)

In [163]:
len(result)

36

In [164]:
def process(raw_string):
    processed_string = raw_string.replace('\n', '').replace('\'', '').replace(
        '"{sentence"', '{"sentence"'
    ).replace('"{', '{').replace('}"', '}').replace('""', '"')
    
    # 加上外层的双引号
    # processed_string = processed_string[1:-1]
    return processed_string
    
new_result = []
for i in range(len(result)):
    # print(i)
    item = result[i]
    
    try:
        item = json.loads(item)
    except:
        try:
            item = json.loads(process(item))
        except:
            p = re.compile(r'\{(.*?)\}')
            matches = p.findall(item)
            item = [json.loads('{'+m+'}}') for m in matches]
    item = [json.loads(i) if isinstance(i, str) else i for i in item]
    new_result.append(item)

In [165]:
len(new_result)

36

In [166]:
df["augment"] = new_result

In [167]:
df.shape

(36, 8)

In [168]:
df_exploded = df.explode("augment")

In [169]:
df_exploded = df_exploded.reset_index(drop=True)

In [170]:
df_exploded['question'] =  df_exploded['augment'].apply(lambda x: x["sentence"].replace("[", "").replace("]", ""))

In [171]:
error_indices = []
for i in range(df_exploded.shape[0]):
    replace = df_exploded['augment'].iloc[i]["replace"]
    for key in replace:
        if key.find("[")>=0:
            error_indices.append(i)
            break 

In [172]:
df_exploded.shape

(239, 9)

In [173]:
df_exploded = df_exploded.drop([df_exploded.index[i] for i in error_indices])

In [174]:
df_exploded.shape

(239, 9)

In [175]:
df_exploded.gen.iloc[0]

{'question': 'm1和h1的面板操作流程是什么？',
 'prompt': [{'primary_value': 'm1',
   'key': '面板操作流程',
   '面板操作流程': '①选择模式：洗烘/洗涤/烘干\n②选定程序：混合/快速/自定义等\n③调节程序状态：转速/漂洗次数/温度\n④启动程序'},
  {'primary_value': 'h1',
   'key': '面板操作流程',
   '面板操作流程': '①选择模式：洗烘/洗涤/烘干\n②选定程序：标准/快速等\n③调节程序状态：干衣度/转速/漂洗次数/温度\n④启动程序'}],
 'replace': {'[问询词0]': 'm1', '[问询词1]': 'h1', '[关键词0]': '面板操作流程'}}

In [176]:
ner_list = []
for i in range(df_exploded.shape[0]):
    replace = df_exploded['augment'].iloc[i]["replace"]
    primaries = [j['primary_value'] for j in df_exploded['gen'].iloc[i]['prompt']]
    flag = False
    entity = []
    for key in replace:
        if replace[key] in primaries:
            print(i)
            flag = True
            break 
        entity.append(replace[key])
    if flag:
        print(i, 'type1 error')
        ner_list.append(np.nan)
        continue
    question =  df_exploded['question'].iloc[i]
    ner = {}
    for item in entity:
        matches = list(re.finditer(item, question))
        loc = [[j.start(), j.end()-1] for j in matches]
        ner[item] = loc 
    ner_list.append({"name": ner})

In [177]:
df_exploded.iloc[10]

cat                                                          washing
gen                {'question': 'm1和h1的面板操作流程是什么？', 'prompt': [{'...
template                                    [问询词0]和[问询词1]的[关键词0]是什么？
cat_num                                                            1
type                                 gen_same_keywords_for_models_v2
final_prompt       {"template": "[问询词0]和[问询词1]的[关键词0]是什么？", "repl...
primary_key_num                                                    2
augment            {'sentence': '关于m1和h1的[操作过程]可以介绍一下吗？', 'replac...
question                                        关于m1和h1的操作过程可以介绍一下吗？
Name: 10, dtype: object

In [178]:
df_exploded["ner_list"] = ner_list

In [179]:
df_exploded.shape

(239, 10)

In [180]:
df_exploded = df_exploded[df_exploded["ner_list"].notnull()]

In [181]:
df_exploded.shape

(239, 10)

In [182]:
 df_exploded[["question", "ner_list"]].head()

,question,ner_list
0,请问m1和h1的操作方法是怎样的？,"{'name': {'操作方法': [[8, 11]]}}"
1,你能告诉我m1和h1的操作面板流程吗？,"{'name': {'操作面板流程': [[11, 16]]}}"
2,我想知道m1和h1的操作方式？,"{'name': {'操作方式': [[10, 13]]}}"
3,可以介绍一下m1和h1的面板操控过程吗？,"{'name': {'面板操控过程': [[12, 17]]}}"
4,m1和h1的操控操作步骤是怎样的？,"{'name': {'操控操作步骤': [[6, 11]]}}"


In [722]:
a = df_exploded[(df_exploded["type"]=="gen_different_keywords_different_models_v2"
            )&(df_exploded["cat_num"]==1)|(df_exploded["type"]=="gen_different_keywords_different_models_v2"
            )&(df_exploded["cat_num"]==2)].augment
for i in range(a.shape[0]):
    print(a.iloc[i])

{'sentence': '请问m1和h1有多少[浸泡功能]，h1neo和h1的[控制方式]是什么？', 'replace': {'浸泡功能': '浸泡功能', '控制方式': '控制方式'}}
{'sentence': '能介绍一下m1和h1所含的[浸泡功能]，以及h1neo和h1的[控制方式]吗？', 'replace': {'浸泡功能': '浸泡功能', '控制方式': '控制方式'}}
{'sentence': 'm1和h1的[浸泡功能]、h1neo和h1的[控制方式]是什么呢？', 'replace': {'浸泡功能': '浸泡功能', '控制方式': '控制方式'}}
{'sentence': 'm1和h1的[浸泡功能]，以及h1neo和h1的[控制方式]能告诉我吗？', 'replace': {'浸泡功能': '浸泡功能', '控制方式': '控制方式'}}
{'sentence': '我想了解一下m1和h1的[浸泡功能]以及h1neo和h1的[控制方式]。', 'replace': {'浸泡功能': '浸泡功能', '控制方式': '控制方式'}}
{'sentence': '请告诉我m1和h1有什么[浸泡功能]以及h1neo和h1的[控制方式]。', 'replace': {'浸泡功能': '浸泡功能', '控制方式': '控制方式'}}
{'sentence': '我能了解一下m1和h1的[浸泡功能]，以及h1neo和h1的[控制方式]吗？', 'replace': {'浸泡功能': '浸泡功能', '控制方式': '控制方式'}}
{'sentence': '可以告诉我m1和h1的[浸泡功能]，还有h1neo和h1的[控制方式]是什么吗？', 'replace': {'浸泡功能': '浸泡功能', '控制方式': '控制方式'}}
{'sentence': '能否告知m1和h1的具备哪些[浸泡功能]，以及h1neo和h1的[控制方式]？', 'replace': {'浸泡功能': '浸泡功能', '控制方式': '控制方式'}}
{'sentence': '请介绍一下m1和h1的[浸泡功能]以及h1neo和h1的[控制方式]。', 'replace': {'浸泡功能': '浸泡功能', '控制方式': '控制方式'}}
{'sentence':

In [183]:
df_exploded.augment.apply(lambda x: len(x["replace"])==0).sum()

0

In [184]:
df_exploded.augment.apply(lambda x: x["sentence"].find("[")<0).sum()

1

In [55]:
import re

def wrap_b_in_brackets(a, b):
    # 使用re.escape来处理特殊字符
    b_escaped = re.escape(b)
    # 使用正则表达式替换所有的b为[b]
    result = re.sub(b_escaped, f'[{b}]', a)
    return result

def wrap(x):
    sentence = x["sentence"]
    do_set = set()
    for key in x["replace"]:
        if x["replace"][key] not in do_set:
            do_set.add(x["replace"][key])
            sentence = wrap_b_in_brackets(sentence, x["replace"][key])
    return {"sentence": sentence, "replace": x["replace"]}

In [726]:
a = df_exploded.loc[df_exploded.augment.apply(lambda x: x["sentence"].find("[")<0), "augment"
].apply(lambda x: wrap(x))
for i in range(a.shape[0]):
    print(a.iloc[i])

{'sentence': '请问h1neo, m1和h1的[柔顺剂容量]和[电机类型]是什么？', 'replace': {'柔顺剂容量': '柔顺剂容量', '电机类型': '电机类型'}}


In [56]:
df_exploded.loc[df_exploded.augment.apply(lambda x: x["sentence"].find("[")<0), "augment"
] = df_exploded.loc[df_exploded.augment.apply(lambda x: x["sentence"].find("[")<0), "augment"
].apply(lambda x: wrap(x))

In [57]:
df_exploded["question_raw"] = df_exploded['augment'].apply(lambda x: x["sentence"])

In [58]:
df_exploded = df_exploded.reset_index(drop=True)
df_exploded["index"] = range(df_exploded.shape[0])

In [730]:
json_list = ["gen", "augment", "ner_list"]
for col in json_list:
    df_exploded[col] = df_exploded[col].apply(lambda x: json.dumps(x, ensure_ascii=False))

In [731]:
output_cols = [col for col in df_exploded.columns if col != "ner_list"]
df_exploded[output_cols].to_csv("/data/dataset/kefu/gpt4_template_v2/generated.csv", index=None)

In [732]:
# 将所有模板收集的数据放在一起，形成测试集合
# 1. v1版本的所有单一品类
# 2. v2版本

In [414]:
df_exploded_v1 = pd.read_csv("/data/dataset/kefu/gpt4_template/generated.csv")

In [415]:
df_exploded_v2 = pd.read_csv("/data/dataset/kefu/gpt4_template_v2/generated.csv")

In [416]:
df_exploded_all = pd.concat([df_exploded_v1, df_exploded_v2], axis=0).reset_index(drop=True)

In [417]:
df_exploded = df_exploded_all.copy()
df_exploded["augment"] = df_exploded["augment"].apply(lambda x: json.loads(x))
df_exploded["final_prompt"] = df_exploded["final_prompt"].apply(lambda x: json.loads(x))
df_exploded['gen'] = df_exploded["gen"].apply(lambda x: json.loads(x))

In [418]:
df_exploded.shape

(1614, 11)

In [419]:
df_exploded["augment"].apply(lambda x: len(x["replace"])==0).sum()

0

In [420]:
df_exploded["augment"].apply(lambda x: x["sentence"].find("[")<0).sum()

0

In [421]:
error_indices = []
for i in range(df_exploded.shape[0]):
    replace = df_exploded['augment'].iloc[i]["replace"]
    for key in replace:
        if key.find("[")>=0:
            error_indices.append(i)
            break 
df_exploded = df_exploded.drop([df_exploded.index[i] for i in error_indices])

In [422]:
df_exploded.shape

(1614, 11)

In [423]:
ner_list = []
for i in range(df_exploded.shape[0]):
    replace = df_exploded['augment'].iloc[i]["replace"]
    primaries = [j['primary_value'] for j in df_exploded['gen'].iloc[i]['prompt']]
    flag = False
    entity = []
    for key in replace:
        if replace[key] in primaries:
            print(i)
            flag = True
            break 
        entity.append(replace[key])
    if flag:
        print(i, 'type1 error')
        ner_list.append(np.nan)
        continue
    question =  df_exploded['question'].iloc[i]
    ner = {}
    for item in entity:
        matches = list(re.finditer(item, question))
        loc = [[j.start(), j.end()-1] for j in matches]
        ner[item] = loc 
    ner_list.append({"name": ner})
df_exploded["ner_list"] = ner_list
df_exploded = df_exploded[df_exploded["ner_list"].notnull()]

In [424]:
df_exploded.shape

(1614, 12)

In [425]:
error_indices = []
for i in range(df_exploded.shape[0]):
    replace = df_exploded.augment.iloc[i]["replace"]
    question_raw =  df_exploded.augment.iloc[i]["sentence"]
    for key in replace:
        if "["+replace[key]+"]" not in question_raw:
            print(i, question_raw, replace)
            error_indices.append(i)
            break

42 g20标准版有哪些[热水洗布]的功能？ {'热水洗布': '热水洗布的功能'}
43 您能告诉我g20标准版的[热水洗布]操作流程吗？ {'热水洗布': '热水洗布操作流程'}
44 我想了解一下g20标准版的[热水洗布]特性。 {'热水洗布': '热水洗布特性'}
63 你能告诉我g20标准版的[机身维护]事宜吗？ {'机身维护': '机身维护事宜'}
133 你能告诉我g20标准版的[烘干功能]的温度是怎样的吗？ {'烘干温度': '烘干功能的温度'}
134 请问g20标准版的[烘干模式]下的温度是什么？ {'烘干温度': '烘干模式下的温度'}
159 能否告诉我[g20标准版]软件算法是怎样的？ {'软件算法': '软件算法'}
234 g20标准版是由哪个[品牌]制作的？ {'品牌': '制作品牌'}
235 你能告诉我g20标准版是哪个[品牌]出品的吗？ {'品牌': '出品品牌'}
236 g20标准版的制造[品牌]是哪家？ {'品牌': '制造品牌'}
420 a10ultra具备何种[设备共享]能力？ {'设备共享': '设备共享能力'}
675 h1的[中途添衣]功能具体是怎样的？ {'中途添衣': '中途添衣功能'}
678 请问h1有[急救开门]功能吗？ {'紧急开门': '急救开门功能'}
680 h1是否配备了[紧急开启门]功能？ {'紧急开门': '紧急开启门功能'}
727 您能告诉我g20标准版在充电和烘干状态下的[额定功率]，以及p10s标准版在集尘状态下的[额定功率]分别是什么吗？ {'额定输入功率（充电 + 烘干状态）': '在充电和烘干状态下的额定功率', '额定输入功率（集尘状态）': '在集尘状态下的额定功率'}
728 你能告诉我g20标准版在充电烘干的状态下[功率]和p10s标准版在开启集尘状态的时候的[功率]吗？ {'额定输入功率（充电 + 烘干状态）': '在充电烘干的状态下功率', '额定输入功率（集尘状态）': '在开启集尘状态的时候的功率'}
829 请问g20标准版的[烘干功能的温度]和p10s标准版[的漫扫和拖地模式]分别是什么样的？ {'烘干温度': '烘干功能的温度', '扫拖模式': '漫扫和拖地模式'}
835 能否告知一下，g20标准版的充电方式和p10s标准版的[预约功能]到底如何？

In [426]:
df_exploded = df_exploded.drop([df_exploded.index[i] for i in error_indices])

In [427]:
df_exploded["index_all"] = range(df_exploded.shape[0])
df_exploded["index_all"] = df_exploded["index_all"].astype(str)

In [428]:
def tagging(x, tag2id):
    new_list = []
    for i in x:
        new_list.append(tag2id[i])
    return new_list

df_exploded["bio"] = df_exploded[["question", "ner_list"]].apply(lambda x: bio_tagging(x["question"], x["ner_list"]), axis=1)
df_exploded["bio_words"] = df_exploded["bio"].apply(lambda x: x[0])
df_exploded["bio_tags"] = df_exploded["bio"].apply(lambda x: x[1])
df_exploded = df_exploded.drop("bio", axis=1)
tag2id = {'O': 0, 'B-name': 1, 'I-name': 2}
df_exploded["bio_tags_id"] = df_exploded["bio_tags"].apply(lambda x:tagging(x, tag2id))

In [429]:
df_exploded.final_prompt.iloc[0]

{'template': '[问询词0]的[关键词0]是什么？',
 'replace': {'[问询词0]': 'g20标准版', '[关键词0]': '商品链接'}}

In [431]:
# 这个是用来做embedding召回模型的测试集
# true_entity_list = []
# pred_entity_list = []
# for i in range(df_exploded.shape[0]):
#     for true_entity in df_exploded["augment"].iloc[i]["replace"]:
#         true_entity_list.append(true_entity)
#         pred_entity_list.append(df_exploded["augment"].iloc[i]["replace"][true_entity])
# df_entity = pd.DataFrame()
# df_entity["true_entity"] = true_entity_list
# df_entity["pred_entity"] = pred_entity_list
# df_entity.to_csv("/data/dataset/kefu/entity_link_test.csv", index=None)

In [433]:
# 这个是给业务人员打标用的测试集
# df_exploded["crafted_question"] = df_exploded["augment"].apply(lambda x: x["sentence"])
# df_exploded["real_entities"] = df_exploded["augment"].apply(lambda x: x["replace"])
# df_exploded["set_name"] = "test"
# output_cols = ["set_name", "index_all", "cat", "template", "crafted_question", "real_entities"]
# df_exploded[output_cols].to_excel("/data/dataset/kefu/for_label_test.xlsx", index=None)

In [434]:
features = Features({'ner_tags': Sequence(ClassLabel(num_classes=3, names=['O', 'B-name', 'I-name'])),
                     'tokens': Sequence(Value(dtype='string')),
                     "cat_num": ClassLabel(num_classes=4, names=[0,1,2,3]),
                     "index_all": Value(dtype='string')})
raw_datasets = Dataset.from_pandas(df_exploded[["bio_words", "bio_tags_id", "cat_num", "index_all"]].rename(columns={"bio_words": "tokens", "bio_tags_id": "ner_tags"}),
                    features=features, preserve_index=False)

In [435]:
raw_datasets.save_to_disk("/data/dataset/kefu/ner_from_template_augment_test_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1567 [00:00<?, ? examples/s]

In [440]:
# 这个是csv形式的测试集
# json_list = ["gen", "final_prompt", "augment", "ner_list", "bio_words", "bio_tags", "bio_tags_id"]
# for col in json_list:
#     df_exploded[col] = df_exploded[col].apply(lambda x: json.dumps(x, ensure_ascii=False))
# df_exploded.to_csv("/data/dataset/kefu/table_qa.csv", index=None)

In [413]:
# 训练集制作
# 由于所有转义表达的词语都很少有重复的，这样划分的train test会过拟合，
# 因此尝试尽量不改变实体，只改句子结构，形成训练集

In [441]:
df_exploded_v1 = pd.read_csv("/data/dataset/kefu/gpt4_template/generated.csv")
df_exploded_v2 = pd.read_csv("/data/dataset/kefu/gpt4_template_v2/generated.csv")
df_exploded = pd.concat([df_exploded_v1, df_exploded_v2], axis=0).reset_index(drop=True)

In [442]:
df_exploded.augment = df_exploded.augment.apply(lambda x: json.loads(x))

In [443]:
df_exploded.augment.apply(lambda x: len(x["replace"])==0).sum()

0

In [444]:
df_exploded.question_raw.apply(lambda x: x.find("[")<0).sum()

0

In [445]:
for i in range(df_exploded[df_exploded.augment.apply(lambda x: x["sentence"].find("[")<0)].shape[0]):
    print(df_exploded[df_exploded.augment.apply(lambda x: x["sentence"].find("[")<0)].iloc[i].augment)

In [776]:
prompt = """
输入的句子中用[]符号括起来的部分是命名实体，请按照以下步骤执行：
1. 对整个句子调整句子结构并保持意思不变且新句子语法通顺，上述调整中，用[]符号括起来的部分作为一个整体参与调整，内部保持不变
2. 对于调整后的句子，去除[]符号，然后检查语法是否通顺，如果是则还原[]符号来标记命名实体，否则转到步骤3
3. 重新对原始输入的句子做转义表达以保证语法通顺，最后提取与原始输入的句子中[]符号括起来的命名实体对应的新实体，并用[]符号括起来

最好的例子（步骤1执行完后保持原始命名实体不变且句子语法通顺）：
输入：
请问g20是否有[上下水功能]？
输出：
{{"sentence": "我想了解下g20的[上下水功能]？", "replace": {{"上下水功能": "上下水功能"}}}}

次好的例子（步骤1执行完后，由于句子语法不通顺而对原始命名实体做了转义，但是最后句子是通顺的）：
输入：
g20[是否支持上下水]？
输出：
{{"sentence": "g20[支持上下水]吗？", "replace": {{"是否支持上下水": "支持上下水"}}}}

不好的例子（保持原始命名实体不变但句子语法不通顺）：
输入：
g20的[操作方法]是怎么的？
输出：
{{"sentence": "m1和h1的[操作方法]具体是怎么操作的？", "replace": {{"操作方法": "操作方法"}}}}


请对每个输入给出{}种不同的说法, 每种说法都要按照上述3个步骤依次做一遍，每种说法对应一个json，放在列表里面返回, 除此之外不要给出任务其他的信息，即如果要求给出n种说法，则输出
[json_0, json_1, json_2, ..., json_n-1]，如果要给出1种说法，则输出[json_0]

输入：
{}
输出：
"""

In [446]:
df_exploded["type"].value_counts()

type
gen_same_keywords_for_models                  696
gen_different_keywords_different_models       679
gen_same_keywords_for_models_v2               120
gen_different_keywords_different_models_v2    119
Name: count, dtype: int64

In [777]:
result = []
prev_time = time.time()
for i in range(df_exploded.shape[0]):
    if i % 10 == 0:
        print(i)
        cur_time = time.time()
        print(cur_time-prev_time)
        prev_time = cur_time
    if df_exploded["type"].iloc[i].find("_v2")>=0:
        n = 3
    else:
        n = 1
    item = generate_answer(prompt, df_exploded["question_raw"].iloc[i], n)
    result.append(item)
    joblib.dump(item, "/data/dataset/kefu/gpt4_template_all_2/{}.jsonl".format(i))

0
0.00029277801513671875
10
30.172164916992188
20
21.10416555404663
30
30.362523317337036
40
46.01581001281738
50
31.858498573303223
60
28.333003520965576
70
23.805840969085693
80
25.66316795349121
90
28.82840919494629
100
24.411174774169922
110
28.753122568130493
120
28.996712923049927
130
30.84315299987793
140
29.060205459594727
150
30.57560110092163
160
24.969565868377686
170
23.712473392486572
180
26.84784245491028
190
28.322980165481567
200
24.32093834877014
210
21.717167139053345
220
26.594087600708008
230
28.41340970993042
240
24.618606090545654
250
28.30187201499939
260
25.46821093559265
270
27.963725805282593
280
31.466816663742065
290
29.47443389892578
300
22.322694301605225
310
23.533947944641113
320
28.271371841430664
330
29.30790948867798
340
27.0170841217041
350
29.732333183288574
360
33.33962845802307
370
37.092719316482544
380
31.706613063812256
390
31.172128677368164
400
25.46167826652527
410
25.534046411514282
420
25.431107997894287
430
25.74759006500244
440
28.764384

In [447]:
df_exploded.shape

(1614, 11)

In [448]:
result = []
for i in range(df_exploded.shape[0]):
    item = joblib.load("/data/dataset/kefu/gpt4_template_all_2/{}.jsonl".format(i))
    result.append(item)

In [449]:
len(result)

1614

In [450]:
result[0]

'[{"sentence": "能否给我一个g20标准版的[商品访问链接]？", "replace": {"商品访问链接": "商品访问链接"}}]'

In [451]:
json.loads(result[1566])

[{'sentence': '关于g10spure和p10spro上下水版的[拖地]方法和p10以及g10spro的[最大高度]能到达多少，你能告诉我吗？',
  'replace': {'拖地': '拖地', '最大高度': '最大高度'}},
 {'sentence': '我想知道g10spure和p10spro上下水版是如何进行[拖地]，还有p10和g10spro的[最大高度]达到了多少？',
  'replace': {'拖地': '拖地', '最大高度': '最大高度'}},
 {'sentence': '你能告诉我g10spure和p10spro上下水版的[拖地]操作方式，以及p10和g10spro的[最大高度]具体数值吗？',
  'replace': {'拖地': '拖地', '最大高度': '最大高度'}}]

In [452]:
def process(raw_string):
    processed_string = raw_string.replace('\n', '').replace('\'', '').replace(
        '"{sentence"', '{"sentence"'
    ).replace('"{', '{').replace('}"', '}').replace('""', '"')
    
    # 加上外层的双引号
    # processed_string = processed_string[1:-1]
    return processed_string

def preprocess(x):
    if (x.find("[") >= 0) and (x[x.find("[")+1] != "{"):
        return "[{"+x[1:-1]+"}]"
    else:
        return x 
    
new_result = []
error_list = []
for i in range(len(result)):
    item = result[i]
    try:
        try:
            item = json.loads(item)
        except:
            try:
                item = json.loads(process(item))
            except:
                item = json.loads(preprocess(item))
        item = [json.loads(i) if isinstance(i, str) else i for i in item]
    except:
        item = np.nan
        error_list.append(i)
    new_result.append(item)

In [453]:
len(new_result)

1614

In [454]:
df_exploded["augment_sentence_only"] = new_result

In [455]:
df_exploded = df_exploded[df_exploded["augment_sentence_only"].notnull()]

In [456]:
df_exploded.shape

(1613, 12)

In [457]:
df_exploded[["type", "index"]].duplicated().sum()

0

In [458]:
df_exploded = df_exploded.explode("augment_sentence_only")

In [459]:
df_exploded.shape

(2091, 12)

In [460]:
df_exploded['augment_sentence_only'].apply(lambda x: len(x["replace"])==0).sum()

0

In [461]:
df_exploded['augment_sentence_only'].apply(lambda x: x["sentence"].find("[")<0).sum()

1

In [462]:
a = df_exploded.loc[df_exploded.augment_sentence_only.apply(lambda x: x["sentence"].find("[")<0), "augment_sentence_only"
]
for i in range(a.shape[0]):
    print(a.iloc[i])

{'sentence': '关于电器基站的功能，g20标准版有什么？', 'replace': {'关于电器基站的功能': '关于电器基站的功能'}}


In [463]:
df_exploded = df_exploded.loc[df_exploded.augment_sentence_only.apply(lambda x: x["sentence"].find("[")>=0)]

In [464]:
df_exploded.shape

(2090, 12)

In [465]:
df_exploded = df_exploded.reset_index(drop=True)
error_indices = []
for i in range(df_exploded.shape[0]):
    replace = df_exploded['augment_sentence_only'].iloc[i]["replace"]
    for key in replace:
        if key.find("[")>=0:
            error_indices.append(i)
            break 
df_exploded = df_exploded.drop([df_exploded.index[i] for i in error_indices])

In [466]:
df_exploded.shape

(2090, 12)

In [467]:
df_exploded["augment_question"] = df_exploded['augment_sentence_only'].apply(lambda x: x["sentence"].replace('[', "").replace(']', ""))

In [468]:
ner_list = []
for i in range(df_exploded.shape[0]):
    replace = df_exploded['augment_sentence_only'].iloc[i]["replace"]
    entity = []
    for key in replace:
        entity.append(replace[key])
    question =  df_exploded['augment_question'].iloc[i]
    ner = {}
    for item in entity:
        matches = list(re.finditer(item, question))
        loc = [[j.start(), j.end()-1] for j in matches]
        ner[item] = loc 
    ner_list.append({"name": ner})

In [469]:
df_exploded["ner_list"] = ner_list

In [470]:
df_exploded = df_exploded[df_exploded["ner_list"].notnull()]

In [471]:
df_exploded.shape

(2090, 14)

In [472]:
df_exploded[["augment_question", "ner_list"]].head()

,augment_question,ner_list
0,能否给我一个g20标准版的商品访问链接？,"{'name': {'商品访问链接': [[13, 18]]}}"
1,你能提供g20标准版的购买链接吗？,"{'name': {'购买链接': [[11, 14]]}}"
2,你知道在哪里我可以找到g20标准版的在线购买页面链接吗？,"{'name': {'在线购买页面链接': [[18, 25]]}}"
3,你能解答下g20标准版的平台是什么吗？,"{'name': {'平台': [[12, 13]]}}"
4,你能告诉我g20标准版使用的系统平台是什么吗？,"{'name': {'系统平台': [[14, 17]]}}"


In [473]:
df_exploded["ner_list"].apply(lambda x: len(x["name"])==0).sum()

0

In [474]:
df_exploded = df_exploded[df_exploded["ner_list"].apply(lambda x: len(x["name"])>0)]

In [475]:
df_exploded.shape

(2090, 14)

In [476]:
df_exploded["bio"] = df_exploded[["augment_question", "ner_list"]].apply(
    lambda x: bio_tagging(x["augment_question"], x["ner_list"]), axis=1)

In [477]:
df_exploded["bio_words"] = df_exploded["bio"].apply(lambda x: x[0])
df_exploded["bio_tags"] = df_exploded["bio"].apply(lambda x: x[1])
df_exploded = df_exploded.drop("bio", axis=1)

In [478]:
tag2id = {'O': 0, 'B-name': 1, 'I-name': 2}

In [479]:
def tagging(x, tag2id):
    new_list = []
    for i in x:
        new_list.append(tag2id[i])
    return new_list

In [480]:
df_exploded["bio_tags_id"] = df_exploded["bio_tags"].apply(lambda x:tagging(x, tag2id))

In [481]:
error_indices = []
for i in range(df_exploded.shape[0]):
    replace = df_exploded.augment.iloc[i]["replace"]
    question_raw =  df_exploded.augment.iloc[i]["sentence"]
    for key in replace:
        if "["+replace[key]+"]" not in question_raw:
            print(i, question_raw, replace)
            error_indices.append(i)
            break

42 g20标准版有哪些[热水洗布]的功能？ {'热水洗布': '热水洗布的功能'}
43 您能告诉我g20标准版的[热水洗布]操作流程吗？ {'热水洗布': '热水洗布操作流程'}
44 我想了解一下g20标准版的[热水洗布]特性。 {'热水洗布': '热水洗布特性'}
63 你能告诉我g20标准版的[机身维护]事宜吗？ {'机身维护': '机身维护事宜'}
133 你能告诉我g20标准版的[烘干功能]的温度是怎样的吗？ {'烘干温度': '烘干功能的温度'}
134 请问g20标准版的[烘干模式]下的温度是什么？ {'烘干温度': '烘干模式下的温度'}
158 能否告诉我[g20标准版]软件算法是怎样的？ {'软件算法': '软件算法'}
233 g20标准版是由哪个[品牌]制作的？ {'品牌': '制作品牌'}
234 你能告诉我g20标准版是哪个[品牌]出品的吗？ {'品牌': '出品品牌'}
235 g20标准版的制造[品牌]是哪家？ {'品牌': '制造品牌'}
420 a10ultra具备何种[设备共享]能力？ {'设备共享': '设备共享能力'}
675 h1的[中途添衣]功能具体是怎样的？ {'中途添衣': '中途添衣功能'}
678 请问h1有[急救开门]功能吗？ {'紧急开门': '急救开门功能'}
680 h1是否配备了[紧急开启门]功能？ {'紧急开门': '紧急开启门功能'}
727 您能告诉我g20标准版在充电和烘干状态下的[额定功率]，以及p10s标准版在集尘状态下的[额定功率]分别是什么吗？ {'额定输入功率（充电 + 烘干状态）': '在充电和烘干状态下的额定功率', '额定输入功率（集尘状态）': '在集尘状态下的额定功率'}
728 你能告诉我g20标准版在充电烘干的状态下[功率]和p10s标准版在开启集尘状态的时候的[功率]吗？ {'额定输入功率（充电 + 烘干状态）': '在充电烘干的状态下功率', '额定输入功率（集尘状态）': '在开启集尘状态的时候的功率'}
829 请问g20标准版的[烘干功能的温度]和p10s标准版[的漫扫和拖地模式]分别是什么样的？ {'烘干温度': '烘干功能的温度', '扫拖模式': '漫扫和拖地模式'}
835 能否告知一下，g20标准版的充电方式和p10s标准版的[预约功能]到底如何？

In [482]:
df_exploded = df_exploded.drop([df_exploded.index[i] for i in error_indices])

In [483]:
import copy
error_indices = []
true_entity_list = []
pred_entity_list = []
for i in range(df_exploded.shape[0]):
    try:
        for intermediate_true_entity in df_exploded["augment_sentence_only"].iloc[i]["replace"]:
            inv_dict = copy.deepcopy(df_exploded["augment"].iloc[i]["replace"])
            inv_dict = {inv_dict[key]: key for key in inv_dict}
            true_entity_list.append(inv_dict[intermediate_true_entity])
            pred_entity_list.append(df_exploded["augment_sentence_only"].iloc[i]["replace"][intermediate_true_entity])
    except:
        error_indices.append(i)
        print(i, df_exploded["augment_sentence_only"].iloc[i], df_exploded["augment"].iloc[i])

886 {'sentence': '请告知我[g20标准版]的[主刷转速]以及[p10s标准版]是否具备[虚拟墙设定]功能?', 'replace': {'g20标准版': 'g20标准版', '主刷转速': '主刷转速', 'p10s标准版': 'p10s标准版', '虚拟墙设定': '虚拟墙设定'}} {'sentence': '比较下，[g20标准版]的[主刷转速]是多少，以及[p10s标准版]有没有[虚拟墙设定]？', 'replace': {'主刷转速': '主刷转速', '有无虚拟墙': '虚拟墙设定'}}
887 {'sentence': '我想了解一下，[g20标准版]的[主刷旋转速度]是多少，以及[p10s标准版]是否[支持虚拟墙功能]？', 'replace': {'g20标准版': 'g20标准版', '主刷旋转速度': '主刷旋转速度', 'p10s标准版': 'p10s标准版', '支不支持虚拟墙功能': '支持虚拟墙功能'}} {'sentence': '能告诉我[g20标准版]的[主刷旋转速度]以及[p10s标准版]它[支不支持虚拟墙功能]吗？', 'replace': {'主刷转速': '主刷旋转速度', '有无虚拟墙': '支不支持虚拟墙功能'}}
888 {'sentence': '我想了解下[g20标准版]的[主刷旋转速率]以及[p10s标准版]的[虚拟墙功能是否开放]？', 'replace': {'g20标准版': 'g20标准版', '主刷旋转速率': '主刷旋转速率', 'p10s标准版': 'p10s标准版', '虚拟墙功能是否开放': '虚拟墙功能是否开放'}} {'sentence': '请问[g20标准版]的[主刷旋转速率]和[p10s标准版]的[虚拟墙功能是否开放]是什么样的？', 'replace': {'主刷转速': '主刷旋转速率', '有无虚拟墙': '虚拟墙功能是否开放'}}
1241 {'sentence': '你能告诉我[电源线]的长度和[h1neo进水口]的长度吗，这两者都是h1的部分？', 'replace': {'电源线': '电源线', 'h1neo的进水口': 'h1neo进水口'}} {'sentence': '能告诉我h1的[电源线]有多长，以及h1neo的[进水口]有多长吗？', 

In [484]:
df_exploded = df_exploded.drop([df_exploded.index[i] for i in error_indices])

In [485]:
df_exploded = df_exploded.reset_index(drop=True)

In [486]:
df_exploded["index_all"] = range(df_exploded.shape[0])
df_exploded["index_all"] = df_exploded["index_all"].astype(str)

In [487]:
# 这个是用来训练embedding召回模型的训练集
# import copy
# true_entity_list = []
# pred_entity_list = []
# for i in range(df_exploded.shape[0]):
#     for intermediate_true_entity in df_exploded["augment_sentence_only"].iloc[i]["replace"]:
#         inv_dict = copy.deepcopy(df_exploded["augment"].iloc[i]["replace"])
#         inv_dict = {inv_dict[key]: key for key in inv_dict}
#         true_entity_list.append(inv_dict[intermediate_true_entity])
#         pred_entity_list.append(df_exploded["augment_sentence_only"].iloc[i]["replace"][intermediate_true_entity])
# df_entity = pd.DataFrame()
# df_entity["true_entity"] = true_entity_list
# df_entity["pred_entity"] = pred_entity_list
# df_entity.to_csv("/data/dataset/kefu/entity_link.csv", index=None)

In [1105]:
# 这个是给业务人员打标用的训练集
# df_exploded["crafted_question"] = df_exploded["augment_sentence_only"].apply(lambda x: x["sentence"])
# df_exploded["real_entities"] = df_exploded["augment"].apply(lambda x: x["replace"])
# df_exploded["set_name"] = "train"
# output_cols = ["set_name", "index_all", "cat", "template", "crafted_question", "real_entities"]
# df_exploded[output_cols].to_excel("/data/dataset/kefu/for_label.xlsx", index=None)

In [488]:
features = Features({'ner_tags': Sequence(ClassLabel(num_classes=3, names=['O', 'B-name', 'I-name'])),
                     'tokens': Sequence(Value(dtype='string')), 
                     "index_all": Value(dtype='int32')})
features

{'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-name', 'I-name'], id=None), length=-1, id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'index_all': Value(dtype='int32', id=None)}

In [489]:
raw_datasets = Dataset.from_pandas(df_exploded[["bio_words", "bio_tags_id", "index_all"]].rename(columns={"bio_words": "tokens", "bio_tags_id": "ner_tags"}),
                    features=features, preserve_index=False)

In [490]:
raw_datasets.save_to_disk("/data/dataset/kefu/ner_from_template_augment_dataset_2")

Saving the dataset (0/1 shards):   0%|          | 0/2027 [00:00<?, ? examples/s]

In [491]:
# 做NER模型

In [798]:
from datasets import load_dataset, DatasetDict

In [799]:
train_datasets = load_from_disk("/data/dataset/kefu/ner_from_template_augment_dataset_2")
test_datasets = load_from_disk("/data/dataset/kefu/ner_from_template_augment_test_dataset")

In [800]:
test_datasets_split = test_datasets.train_test_split(test_size=0.2, seed=42, shuffle=True, stratify_by_column="cat_num")

In [801]:
# 创建一个新的 DatasetDict
raw_datasets = DatasetDict({
    'train': train_datasets,
    'valid': test_datasets_split['test'],
    'test': test_datasets_split['train'],
    'all': test_datasets
})

In [802]:
ner_feature = raw_datasets['train'].features["ner_tags"]
label_names = ner_feature.feature.names
label_names

['O', 'B-name', 'I-name']

In [803]:
words = raw_datasets['train'][0]["tokens"]
labels = raw_datasets['train'][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

能 否 给 我 一 个 g 2 0 标 准 版 的 商      品      访      问      链      接      ？ 
O O O O O O O O O O O O O B-name I-name I-name I-name I-name I-name O 


In [804]:
from transformers import AutoTokenizer
#tokenizer = AutoTokenizer.from_pretrained('hfl/chinese-bert-wwm')
tokenizer = AutoTokenizer.from_pretrained('/data/dataset/huggingface/hub/bert-base-chinese')
# tokenizer = AutoTokenizer.from_pretrained('/data/dataset/huggingface/hub/bert-base-cased')

In [805]:
tokenizer.is_fast

True

In [806]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 '能',
 '否',
 '给',
 '我',
 '一',
 '个',
 'g',
 '2',
 '0',
 '标',
 '准',
 '版',
 '的',
 '商',
 '品',
 '访',
 '问',
 '链',
 '接',
 '？',
 '[SEP]']

In [807]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [808]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 0, -100]


In [809]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [810]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/1253 [00:00<?, ? examples/s]

In [811]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2027
    })
    valid: Dataset({
        features: ['cat_num', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 314
    })
    test: Dataset({
        features: ['cat_num', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1253
    })
    all: Dataset({
        features: ['cat_num', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1567
    })
})

In [812]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [813]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    1,    2,    2,    2,    2,    2,    0, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            1,    2,    2,    2,    0,    0, -100, -100, -100, -100]])

In [814]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 0, -100]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, -100]


In [815]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-name',
 'I-name',
 'I-name',
 'I-name',
 'I-name',
 'I-name',
 'O']

In [816]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [565]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    '/data/dataset/huggingface/hub/bert-base-chinese',
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at /data/dataset/huggingface/hub/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [817]:
# from transformers import AutoModelForTokenClassification

# model = AutoModelForTokenClassification.from_pretrained(
#     '/workspace/data/private/zhuxiaohai/models/bert_finetuned_ner_augmented/',
#     id2label=id2label,
#     label2id=label2id,
# )

In [818]:
model.config.num_labels

3

In [819]:
import evaluate

metric = evaluate.load("seqeval")

In [820]:
import numpy as np
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [821]:
from transformers import TrainingArguments
args = TrainingArguments(
    "/workspace/data/private/zhuxiaohai/models/bert-finetuned-ner-augmented",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=8,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    per_device_train_batch_size=8,
    push_to_hub=False,
)

In [822]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [571]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.075186,0.819765,0.870550,0.844395,0.974440
2,0.091600,0.059735,0.877405,0.906611,0.891769,0.980501
3,0.091600,0.058424,0.896163,0.917707,0.906807,0.983261
4,0.030700,0.056268,0.903576,0.922792,0.913083,0.985428
5,0.030700,0.062229,0.909747,0.932039,0.920758,0.986176
6,0.009800,0.057573,0.923007,0.936662,0.929784,0.988110
7,0.009800,0.059289,0.924374,0.938049,0.931161,0.988394
8,0.005000,0.059052,0.926061,0.938049,0.932017,0.988394


TrainOutput(global_step=2032, training_loss=0.03375346463092962, metrics={'train_runtime': 235.9954, 'train_samples_per_second': 68.713, 'train_steps_per_second': 8.61, 'total_flos': 601206914021418.0, 'train_loss': 0.03375346463092962, 'epoch': 8.0})

In [572]:
trainer.evaluate(tokenized_datasets["valid"])

{'eval_loss': 0.0807223990559578,
 'eval_precision': 0.9032258064516129,
 'eval_recall': 0.9242718446601942,
 'eval_f1': 0.9136276391554702,
 'eval_accuracy': 0.9841337000211551,
 'eval_runtime': 1.0595,
 'eval_samples_per_second': 296.376,
 'eval_steps_per_second': 37.755,
 'epoch': 8.0}

In [573]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.05905203893780708,
 'eval_precision': 0.9260611592879964,
 'eval_recall': 0.938049006010171,
 'eval_f1': 0.9320165365181443,
 'eval_accuracy': 0.9883936861652739,
 'eval_runtime': 4.9372,
 'eval_samples_per_second': 253.786,
 'eval_steps_per_second': 31.799,
 'epoch': 8.0}

In [823]:
df_exploded = pd.read_csv("/data/dataset/kefu/table_qa.csv")

In [824]:
df_exploded.shape

(1567, 16)

In [825]:
tokenized_datasets["all"]

Dataset({
    features: ['cat_num', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1567
})

In [826]:
a = trainer.predict(tokenized_datasets["all"]).predictions

In [827]:
df_exploded["pred"] = np.argmax(a, axis=-1).tolist()
df_exploded["labels"] = tokenized_datasets["all"]["labels"]

In [828]:
def compute_groupby(df):
    labels = df["labels"].tolist()
    predictions = df["pred"].tolist()
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return pd.Series({
        "num": df.shape[0],
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    })

In [835]:
stat = df_exploded.groupby(["type", "cat_num"]).apply(lambda x: compute_groupby(x))

/tmp/ipykernel_1340229/569249638.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stat = df_exploded.groupby(["type", "cat_num"]).apply(lambda x: compute_groupby(x))


In [834]:
stat

num  precision  \
type                                       primary_key_num                     
gen_different_keywords_different_models    2                650.0   0.946483   
gen_different_keywords_different_models_v2 4                 56.0   0.934211   
                                           6                 59.0   0.861925   
gen_same_keywords_for_models               1                682.0   0.867898   
gen_same_keywords_for_models_v2            2                 60.0   1.000000   
                                           3                 60.0   0.983333   

                                                              recall  \
type                                       primary_key_num             
gen_different_keywords_different_models    2                0.953775   
gen_different_keywords_different_models_v2 4                0.950893   
                                           6                0.880342   
gen_same_keywords_for_models               1                0.894583   
gen_same_keywords_for_models_v2            2                1.000000   
                                           3                0.983333   

                                                                  f1  accuracy  
type                                       primary_key_num                      
gen_different_keywords_different_models    2                0.950115  0.990992  
gen_different_keywords_different_models_v2 4                0.942478  0.990590  
                                           6                0.871036  0.979114  
gen_same_keywords_for_models               1                0.881038  0.980300  
gen_same_keywords_for_models_v2            2                1.000000  1.000000  
                                           3                0.983333  0.996833

In [582]:
from transformers import pipeline
# Replace this with your own checkpoint
model_checkpoint = "/workspace/data/private/zhuxiaohai/models/bert_finetuned_ner_augmented"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)

In [583]:
df_exploded["ner"] = df_exploded["question"].apply(lambda x: token_classifier(x))

In [584]:
df_exploded["ner_word"] = df_exploded["ner"].apply(lambda x: [i["word"] for i in x])

In [585]:
df_exploded[(df_exploded["type"]=="gen_different_keywords_different_models_v2"
            )&(df_exploded["cat_num"]==1)][["question_raw", "ner_word"]]

,question_raw,ner_word
1452,请问m1和h1有多少[浸泡功能]，h1neo和h1的[控制方式]是什么？,"[浸 泡 功 能, 控 制 方 式]"
1453,能介绍一下m1和h1所含的[浸泡功能]，以及h1neo和h1的[控制方式]吗？,"[浸 泡 功 能, 控 制 方 式]"
1454,m1和h1的[浸泡功能]、h1neo和h1的[控制方式]是什么呢？,"[浸 泡 功 能, 控 制 方 式]"
1455,m1和h1的[浸泡功能]，以及h1neo和h1的[控制方式]能告诉我吗？,"[浸 泡 功 能, 控 制 方 式]"
1456,我想了解一下m1和h1的[浸泡功能]以及h1neo和h1的[控制方式]。,"[浸 泡 功 能, 控 制 方 式]"
...,...,...
1518,想这样了解下，m1、h1和h1neo他们的[颜色分类有几种]，[盒子的尺寸]，以及[机器的实...,"[颜 色 分 类 有 几 种, 盒 子 的 尺 寸, 机 器 的 实 际 尺 寸, 断 电 ..."
1519,想请教下，m1、h1和h1neo分别有哪些[颜色分类]，他们[包装的尺寸]和[机器本身的尺寸...,"[哪 些, 颜 色 分 类, 包 装 的 尺 寸, 机 器 本 身 的 尺 寸, 有 断 电..."
1520,请问下，m1、h1和h1neo的[颜色分类]，[包装占用的空间大小]，以及[机器实际的尺寸]...,"[颜 色 分 类, 包 装 占 用 的 空 间 大 小, 机 器 实 际 的 尺 寸, 断 ..."
1521,可以告诉我m1、h1和h1neo的[有什么颜色分类]，[包装尺寸有多大]，[机器的尺寸是多少...,"[有 什 么 颜 色 分 类, 包 装 尺 寸 有 多 大, 机 器 的 尺 寸 是 多 少..."


In [901]:
# NER模型的召回效果

In [23]:
import re
import Levenshtein

In [24]:
dim_df = pd.read_csv("/data/dataset/kefu/dim_df20240315.csv")
dim_df.model = dim_df.model.apply(lambda x: x.replace("版本", "").replace("版", ""))
all_model_list = dim_df.model.tolist()
all_cat_list = dim_df.cat_name.unique().tolist()
all_model_list = [i.replace("版本", "").replace("版", "") for i in all_model_list]

In [77]:
df_exploded = pd.read_csv("/data/dataset/kefu/table_qa.csv")
df_exploded["augment"] = df_exploded["augment"].apply(lambda x: json.loads(x))
df_exploded["gen"] = df_exploded["gen"].apply(lambda x: json.loads(x))
df_exploded["final_prompt"] = df_exploded["final_prompt"].apply(lambda x: json.loads(x))

In [78]:
df_exploded["augment"].apply(lambda x: len(x["replace"])==0).sum()

0

In [79]:
df_exploded["augment"].apply(lambda x: x["sentence"].find("[")<0).sum()

0

In [80]:
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings, HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS, Chroma

In [81]:
df_exploded["model_list"] = df_exploded['gen'].apply(lambda x: [item["primary_value"] for item in x["prompt"]])

In [82]:
schema_set = list(df_all.columns)
exclude_cols = ["平台ID", "商品id", "商品编码", "商品分类", "商品名字", "版本", "商品型号", "primary_key"]
schema_set = [col for col in schema_set if col not in exclude_cols]

In [83]:
docs = []
for index, line in enumerate(schema_set):
    data = {"entity_name": line, 
            "washing": "washing" in filter[line],
            "mopping": "mopping" in filter[line],
            "sweeping": "sweeping" in filter[line],
            "content": line.lower(),
            "ids": index,
           }
    docs.append(data)
docs = pd.DataFrame(docs)


In [70]:
docs.to_csv("data/table_entity.csv", index=None)

In [90]:
# encoder = HuggingFaceEmbeddings(model_name="/data/dataset/huggingface/hub/bge-large-zh-v1.5", 
#                                    encode_kwargs = {'normalize_embeddings': True},
#                                    model_kwargs={'device': 'cuda:0'})
# encoder = HuggingFaceEmbeddings(model_name="/workspace/data/private/zhuxiaohai/models/bge_finetuned_emb_ner_link", 
#                                    encode_kwargs = {'normalize_embeddings': True},
#                                    model_kwargs={'device': 'cuda:0'})
encoder = HuggingFaceEmbeddings(model_name="/workspace/data/private/zhuxiaohai/models/bge_finetuned_emb_ner_link", 
                                   encode_kwargs = {'normalize_embeddings': True},
                                   model_kwargs={'device': 'cuda:0'})

In [86]:
from langchain.vectorstores import Chroma

# Initialize the Chroma client
chroma_client = Chroma()

# List all collections
collections = chroma_client._client.list_collections()


In [87]:
for collection in collections:
    chroma_client._client.delete_collection(collection.name)

In [91]:
# encoder = HuggingFaceBgeEmbeddings(model_name="/data/dataset/huggingface/hub/bge-large-zh-v1.5", 
#                                    encode_kwargs = {'normalize_embeddings': True},
#                                    model_kwargs={'device': 'cuda:0'})

store = Chroma.from_documents([Document(page_content=line, metadata={"id": id,
                                                                    "washing": "washing" in filter[line],
                                                                    "mopping": "mopping" in filter[line],
                                                                    "sweeping": "sweeping" in filter[line],
                                                                     "content": line.lower(),
                                                                   })
 for id, line in enumerate(schema_set)], embedding=encoder, collection_name="my_collection")

In [92]:
# from fastbm25 import fastbm25
# from utils import WordCut
# wc = WordCut()

# bm25 = fastbm25([wc.cut(doc) for doc in schema_set])

In [93]:
def find_blocks(models, keywords):
    models_name_list = [model[0] for model in models] 
    keywords_name_list = [keyword[0] for keyword in keywords]
    positions = sorted(models + keywords, key=lambda x: x[1])
    model_blocks = []
    keyword_blocks = []

    current_model_block = []
    current_keyword_block = []

    for i, (word, start, end) in enumerate(positions):
        if word in models_name_list:
            if (i == 0 or positions[i-1][0] in keywords_name_list):
                if current_model_block:
                    model_blocks.append(current_model_block)
                current_model_block = [(word, start, end)]
            else:
                current_model_block.append((word, start, end))

            if (i == len(positions) - 1 or positions[i+1][0] in keywords_name_list):
                model_blocks.append(current_model_block)
                current_model_block = []
        elif word in keywords_name_list:
            if (i == 0 or positions[i-1][0] in models_name_list):
                if current_keyword_block:
                    keyword_blocks.append(current_keyword_block)
                current_keyword_block = [(word, start, end)]
            else:
                current_keyword_block.append((word, start, end))

            if (i == len(positions) - 1 or positions[i+1][0] in models_name_list):
                keyword_blocks.append(current_keyword_block)
                current_keyword_block = []

    return model_blocks, keyword_blocks

def find_nearest_model_block(keyword, model_blocks):
    keyword_start = keyword[1]
    left_model_block = None
    right_model_block = None

    for block in model_blocks:
        block_end = block[-1][2]
        block_start = block[0][1]

        if block_end <= keyword_start:
            left_model_block = block
        elif block_start >= keyword_start and right_model_block is None:
            right_model_block = block

    if left_model_block:
        return left_model_block
    else:
        return right_model_block

In [112]:
def edit_distance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

def recall_from_edit_distance(word, store, schema_set, recall_n=1, cat=None):
    if cat is None:
        filter = None
    else:
        cat = list(set(cat))
        if len(cat) == 1:
            filter = {cat[0]: True}
        else:
            cat_list = [ {
                        i: True
                    } for i in list(set(cat))]
            filter = {
                "$or": cat_list
            }
    filter_content = {"$and": [{"content": word.lower()}, filter]}\
            if filter else {"content": word.lower()}
    searched = store.similarity_search_with_score(word, k=recall_n, filter=filter_content)
    searched += store.similarity_search_with_score(word, k=recall_n, filter=filter)
    recalls = [(schema_set[i[0].metadata["id"]], i[1]) for i in searched]
    # candidates = [i[0]find_model for i in recalls]
    # edit_lens = [edit_distance(i, word) for i in candidates]
    # return [cand for i, cand in enumerate(candidates) if edit_lens[i] != max(len(candidates[i]), len(word)) and edit_lens[i] <= 4]
    results = [i[0] for i in recalls]
    return results 

def recall_from_edit_distance2(word, store, schema_set, recall_n=3):
    searched = store.top_k_sentence(wc.cut(word.lower()), k=recall_n)
    results = [schema_set[i[1]] for i in searched]
    return results 

def find_min_positive_index(b):
    # 筛选出所有大于0的元素及其索引
    positive_elements = [(index, value) for index, value in enumerate(b) if value > 0]
    
    # 如果没有大于0的元素，返回None
    if not positive_elements:
        return None
    
    # 找到最小的正值及其索引
    min_index, min_value = min(positive_elements, key=lambda x: x[1])
    
    return min_index


def prepare_columns_for_sql_v2(query_body, store, schema_set, all_model_dict, cat_list=None):
    query = query_body["query"]
    model_list = find_model_v2(query, all_model_dict)
    keywords = [(query[entity["start"]:entity["end"]], entity["start"], entity["end"]) for entity in query_body.get("entities", [])]
    # if cat_list is None:
    #     model_blocks, keyword_blocks = find_blocks(model_list, keywords)
    #     meta_keywords = []
    #     name_map = {"洗地机": "mopping", "洗衣机": "washing", "扫地机": "sweeping"}
    #     for keyword in keywords:
    #         nearest_model_block = find_nearest_model_block(keyword, model_blocks)
    #         if nearest_model_block:
    #             words = [word for word, _, _ in nearest_model_block]
    #             cat_list = [all_model_dict[word] for word in words]
    #             cat_list = [name_map[i] for i in cat_list]
    #         else:
    #             cat_list = None
    #         meta_keywords.append(cat_list)
    if cat_list is None:
        name_map = {"洗地机": "mopping", "洗衣机": "washing", "扫地机": "sweeping"}
        cat_list = [all_model_dict[model[0]] for model in model_list]
        cat_list = [name_map[i] for i in cat_list]
        meta_keywords = []
        for keyword in keywords:
            meta_keywords.append(cat_list)
    else:
        meta_keywords = []
        for keyword in keywords:
            meta_keywords.append(cat_list)
        
    all_cols = []
    
    # for keyword in keywords:
    #     if keyword[0] in schema_set:
    #         all_cols.append(keyword[0])
            
    for keyword, cat in zip(keywords, meta_keywords):
        all_cols += recall_from_edit_distance(keyword[0], store, schema_set, cat=cat)

    all_cols_set = []
    for col in all_cols:
        if col not in all_cols_set:
            all_cols_set.append(col)

    return all_cols_set

def get_keywords(model, query):
    entities = model(query)
    return {"query": query, "entities": entities}

In [95]:
from transformers import pipeline
# Replace this with your own checkpoint
model_checkpoint = "/workspace/data/private/zhuxiaohai/models/bert_finetuned_ner_augmented/"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)

In [96]:
# 以下这一段代码是用来造训练embedding召回模型的训练集最终版本的

In [97]:
# df_entity = pd.read_csv("/data/dataset/kefu/entity_link.csv")

In [98]:
# neg = []
# for i in range(df_entity.shape[0]):
#     pred_entity = df_entity.iloc[i]["pred_entity"]
#     true_entity = df_entity.iloc[i]["true_entity"]
#     cat = list(filter[true_entity])
#     # cat = None
#     all_cols = recall_from_edit_distance(pred_entity, store, schema_set, recall_n=10, cat=cat)[2:5]
#     neg.append(all_cols)
# df_entity["neg"] = neg

In [99]:
# df_entity["true_entity"] = df_entity["true_entity"].apply(lambda x: [x])

In [785]:
# recall_from_edit_distance("商品访问链接", store, schema_set, recall_n=10, cat=list({'mopping', 'sweeping', 'washing'}))

['商品链接',
 '采购地',
 '店铺名称',
 '支持APP',
 '平台',
 '产品信息',
 '导航类型',
 '标配清单',
 'Wi-Fi连接',
 '电源线']

In [38]:
def convert_df_to_jsonl(df, filename, query="question_cleaned", pos_col="question_positive", neg_col="question_bge_hard"):
    with open(filename, 'w') as file:
        for _, row in df.iterrows():
            # Constructing the dictionary for each row
            data = {
                "query": row[query],
                "pos": row[pos_col],
                "neg": row[neg_col]
            }
            # Writing the JSON string followed by a newline character to make it JSONL
            file.write(json.dumps(data) + '\n')

In [748]:
# convert_df_to_jsonl(df_entity, '/data/dataset/kefu/bge_finetune_emb_finetuned_ner_link_with_filter.jsonl', 
#                     query="pred_entity", pos_col="true_entity", neg_col="neg")

In [786]:
# df_entity["neg"].apply(lambda x: len(x)!= len(set(x))).sum()

0

In [787]:
# df_entity.head()

,true_entity,pred_entity,neg
0,[商品链接],商品访问链接,"[店铺名称, 支持APP, 平台]"
1,[商品链接],购买链接,"[标配清单, 店铺名称, 产品信息]"
2,[商品链接],在线购买页面链接,"[平台, 支持APP, 店铺名称]"
3,[平台],平台,"[商品链接, APP程序, 设备共享]"
4,[平台],系统平台,"[控制面板程序, 设备共享, 标配]"


In [ ]:
# 以上这一段代码是用来造训练embedding召回模型的训练集最终版本的

In [100]:
df_exploded["type"].value_counts()

type
gen_same_keywords_for_models                  682
gen_different_keywords_different_models       650
gen_same_keywords_for_models_v2               120
gen_different_keywords_different_models_v2    115
Name: count, dtype: int64

In [101]:
df_exploded["ner"] = df_exploded["question"].apply(lambda x: get_keywords(token_classifier, x))

In [113]:
recall_model = store
df_exploded["pred"] = df_exploded[["ner", "cat"]].apply(
    lambda x: prepare_columns_for_sql_v2(x["ner"], recall_model, schema_set, all_model_dict, x["cat"].split(",")), axis=1)

In [103]:
def metric(pred, gt):
    gt_key = []
    for item in gt:
        for key in item["key"].split("|"):
            gt_key.append(key)
    gt_key = set(gt_key)
    pred = set(pred)
    match = gt_key & pred
    if len(pred) > 0:
        precision = len(match)/ len(pred)
    else:
        if len(gt_key) == 0:
            precision = 1
        else:
            precision = 0
    if len(gt_key) > 0:
        recall = len(match) / len(gt_key)
    else:
        recall = 1
    return precision, recall

In [114]:
df_exploded["metric"] = df_exploded[["pred", "gen"]].apply(lambda x: metric(x["pred"], x["gen"]["prompt"]), axis=1)
df_exploded["precision"] = df_exploded["metric"].apply(lambda x: x[0])
df_exploded["recall"] = df_exploded["metric"].apply(lambda x: x[1])

In [117]:
name_map = {"洗地机": "mopping", "洗衣机": "washing", "扫地机": "sweeping"}
test_indices = []
for i in range(df_exploded.shape[0]):
    primary = [j["primary_value"] for j in df_exploded.gen[i]["prompt"]]
    primary = list(set([model.replace("标准版", "").replace("上下水版", "").replace("上下水", "") for model in primary]))
    primary = sorted(primary)
    query = df_exploded.ner.iloc[i]["query"]
    model_list = find_model_v2(query, all_model_dict)
    model_list = [model[0] for model in model_list]
    model_list = list(set([model.replace("标准版", "").replace("上下水版", "").replace("上下水", "") for model in model_list]))
    model_list = sorted(model_list)
    cat_list = list(set([all_model_dict[model] for model in model_list]))
    cat_list = [name_map[cat] for cat in cat_list]
    cat_list = sorted(cat_list)
    raw_cat_list = df_exploded.cat.iloc[i].split(",")
    raw_cat_list = sorted(raw_cat_list)
    raw_key = list(set([j["key"] for j in df_exploded.gen.iloc[i]["prompt"]]))
    new_raw_key = []
    for j in raw_key:
        new_raw_key += j.split("|")
    new_raw_key = list(set(new_raw_key))
    raw_key_cat = [filter[key] for key in new_raw_key]
    set_cat = raw_key_cat[0]
    for j in raw_key_cat[1:]:
        set_cat |= j
    set_cat = sorted(list(set_cat))
    if cat_list != raw_cat_list:
        if len(cat_list) > len(raw_cat_list):
            print(i, query)
            print("model", cat_list)
            print("raw", raw_cat_list)
            print("keycat", set_cat)
            break
    else:
        test_indices.append(i)

In [118]:
def get_gt(gt):
    gt_key = []
    for item in gt:
        for key in item["key"].split("|"):
            gt_key.append(key)
    gt_key = set(gt_key)
    return list(gt_key)

def get_pred(pred):
    return list(set(pred))

test = df_exploded.iloc[test_indices].copy()
test["model_list"] = test["ner"].apply(lambda x: find_model_v2(x["query"], all_model_dict))
test["keywords"] = test["ner"].apply(lambda x: [
    (x["query"][entity["start"]:entity["end"]], entity["start"], entity["end"]) 
    for entity in x.get("entities", [])])
test["true_entity"] = test["gen"].apply(lambda x: get_gt(x["prompt"]))
test["pred_entity"] = test["pred"].apply(lambda x: get_pred(x))

In [119]:
for col in ["keywords", "model_list", "true_entity", "pred_entity"]:
    test[col] = test[col].apply(lambda x: json.dumps(x, ensure_ascii=False))
test[["question", "keywords", "model_list", "true_entity", "pred_entity"]].to_csv("tests/data/data_table_qa.csv", index=None)

In [120]:
name_map = {'洗地机': 'mopping', '洗衣机': 'washing', '扫地机': 'sweeping', "吸尘器": "vacuum"}

In [121]:
dim_df = []
for col in all_model_dict:
    data = {"model": col, "cat": name_map[all_model_dict[col]], "cat_cn": all_model_dict[col]}
    dim_df.append(data)
dim_df = pd.DataFrame(dim_df)
dim_df.to_csv("data/dim_df20240619.csv", index=None)

In [1016]:
df_exploded.groupby(["type", "primary_key_num"]).agg({"precision": [len, "mean"]})

precision          
                                                                 len      mean
type                                       primary_key_num                    
gen_different_keywords_different_models    2                     650  0.940513
gen_different_keywords_different_models_v2 4                      56  0.982993
                                           6                      59  0.929742
gen_same_keywords_for_models               1                     682  0.910802
gen_same_keywords_for_models_v2            2                      60  0.941667
                                           3                      60  0.994444

In [1017]:
df_exploded.groupby(["type", "primary_key_num"]).agg({"recall": [len, "mean"]})

recall          
                                                              len      mean
type                                       primary_key_num                 
gen_different_keywords_different_models    2                  650  0.949231
gen_different_keywords_different_models_v2 4                   56  0.994048
                                           6                   59  0.995763
gen_same_keywords_for_models               1                  682  0.931085
gen_same_keywords_for_models_v2            2                   60  0.944444
                                           3                   60  0.994444

In [1018]:
df_exploded.groupby(["type", "primary_key_num"])["recall"].value_counts("mean")

type                                        primary_key_num  recall  
gen_different_keywords_different_models     2                1.000000    0.906154
                                                             0.500000    0.086154
                                                             0.000000    0.007692
gen_different_keywords_different_models_v2  4                1.000000    0.964286
                                                             0.833333    0.035714
                                            6                1.000000    0.983051
                                                             0.750000    0.016949
gen_same_keywords_for_models                1                1.000000    0.931085
                                                             0.000000    0.068915
gen_same_keywords_for_models_v2             2                1.000000    0.933333
                                                             0.000000    0.050000
                            

In [47]:
df_exploded["precision"].mean()

0.9308034764639742

In [48]:
df_exploded["recall"].mean()

0.9462348436502872

In [1]:
# NL2SQL

In [183]:
prompt = """
请你给我写一个sql从数据库查询型号参数，必须在查询结果中包括所有涉及的型号， 但是你需要根据我的问题从涉及的字段中准确选择而不是简单使用全部字段或者捏造字段。直接给出sql除此之外不要返回任何东西。
1. 表名：df
2. 涉及的型号：{}
3. 涉及的字段：{}

我的问题是：
{}
sql:
"""

In [185]:
def generate_answer3(prompt, model_list, col_list, question, model="gpt-4-8k"): # model = "deployment_name"
    response = client.chat.completions.create(
    model=model, # model = "deployment_name".
    messages=[
        {"role": "user", "content": prompt.format(",".join(model_list), ",".join(col_list), question)},
    ]
    )
    return response.choices[0].message.content

In [187]:
result = []
prev_time = time.time()
for i in range(df_exploded.shape[0]):
    if i % 10 == 0:
        print(i)
        cur_time = time.time()
        print(cur_time-prev_time)
        prev_time = cur_time
    item = generate_answer3(prompt, df_exploded.iloc[i].model_list, df_exploded.iloc[i].pred, df_exploded.iloc[i].question)
    result.append(item)
    joblib.dump(item, "/data/dataset/kefu/gpt4_template_sql/{}.jsonl".format(i))

0
0.00191497802734375
10
16.90106463432312
20
11.905110120773315
30
19.474507808685303
40
19.630595684051514
50
16.00170922279358
60
13.391375541687012
70
22.235890865325928
80
16.160751581192017
90
20.687827587127686
100
13.638507604598999
110
15.120668888092041
120
18.546640872955322
130
20.348158836364746
140
17.136104822158813
150
20.882124662399292
160
18.00317144393921
170
18.182682752609253
180
18.085890769958496
190
21.10001254081726
200
16.112658739089966
210
16.337135553359985
220
13.653748035430908
230
15.926572799682617
240
20.126106023788452
250
19.249960899353027
260
12.346578121185303
270
13.472089052200317
280
17.217100620269775
290
14.071109771728516
300
20.342352151870728
310
15.294995307922363
320
17.525383472442627
330
16.690564393997192
340
18.626447439193726
350
14.869285821914673
360
17.506174087524414
370
14.495913028717041
380
17.11882781982422
390
17.15841245651245
400
16.023648977279663
410
20.38727903366089
420
14.304219007492065
430
13.831573486328125
440
1

In [176]:
result = []
for i in range(df_exploded.shape[0]):
    item = joblib.load("/data/dataset/kefu/gpt4_template_sql/{}.jsonl".format(i))
    result.append(item)

In [190]:
df_exploded["sql"] = result

In [191]:
import re


def clean_field(field):
    return field.strip().strip('`').strip('"').strip("'")

def extract_fields_and_models(sql_query):
    # 提取所有 SELECT 和 FROM 之间的部分
    select_from_pattern = r"SELECT\s+(.*?)\s+FROM"
    select_from_matches = re.findall(select_from_pattern, sql_query, re.DOTALL)
    select_from_fields = [clean_field(field) for match in select_from_matches for field in match.split(',')]

    # 提取所有 WHERE 型号 IN 或 WHERE 型号 = 后面的部分
    where_in_or_equal_pattern = r"WHERE\s+`?型号`?\s*=\s*['\"]([^'\"]*)['\"]|WHERE\s+`?型号`?\s*IN\s*\(([^)]*)\)"
    where_in_or_equal_matches = re.findall(where_in_or_equal_pattern, sql_query, re.DOTALL)
    
    where_fields = []
    for match in where_in_or_equal_matches:
        if match[0]:
            where_fields.append(clean_field(match[0]))
        elif match[1]:
            where_fields.extend([clean_field(model) for model in match[1].split(',')])

    return select_from_fields, where_fields

def process_union_queries(sql_query):
    # 分割 UNION ALL 语句
    union_queries = sql_query.split('UNION ALL')
    
    all_fields = []
    all_models = []
    
    for query in union_queries:
        fields, models = extract_fields_and_models(query)
        all_fields.extend(fields)
        all_models.extend(models)
    
    return all_fields, all_models

In [192]:
df_exploded["sql_extracted"] = df_exploded["sql"].apply(lambda x: process_union_queries(x)[0])
df_exploded["sql_metric"] = df_exploded[["sql_extracted", "gen"]].apply(
    lambda x: metric(x["sql_extracted"], x["gen"]["prompt"]), axis=1)
df_exploded["sql_precision"] = df_exploded["sql_metric"].apply(lambda x: x[0])
df_exploded["sql_recall"] = df_exploded["sql_metric"].apply(lambda x: x[1])

In [217]:
df_exploded[df_exploded["recall"]==1]["sql_recall"].mean()

0.9207221350078493

In [196]:
df_exploded[df_exploded["recall"]==1]["sql_recall"].value_counts("mean")

sql_recall
1.000000    0.879906
0.500000    0.077708
0.000000    0.040031
0.833333    0.002355
Name: proportion, dtype: float64

In [254]:
df_exploded[(df_exploded["recall"]==1)&(df_exploded["sql_recall"]==1)]["sql_precision"].value_counts("mean")

sql_precision
1.000000    0.807851
0.666667    0.067149
0.500000    0.059917
0.333333    0.054752
0.400000    0.006198
0.750000    0.002066
0.285714    0.001033
0.857143    0.001033
Name: proportion, dtype: float64

In [206]:
df_exploded[(df_exploded["recall"]==1)&(df_exploded["sql_recall"]==1)&(df_exploded["sql_precision"]==0.5)].iloc[10].gen

{'question': 'g20标准版的集尘频率是什么？',
 'prompt': [{'primary_value': 'g20标准版', 'key': '集尘频率', '集尘频率': nan}],
 'replace': {'[问询词0]': 'g20标准版', '[关键词0]': '集尘频率'}}

In [207]:
df_exploded[(df_exploded["recall"]==1)&(df_exploded["sql_recall"]==1)&(df_exploded["sql_precision"]==0.5)].iloc[10].ner

{'query': '针对G20标准版来说，你了解其多频集尘功能吗？',
 'entities': [{'entity_group': 'name',
   'score': 0.9779524,
   'word': '多 频 集 尘 功 能',
   'start': 15,
   'end': 21}]}

In [208]:
df_exploded[(df_exploded["recall"]==1)&(df_exploded["sql_recall"]==1)&(df_exploded["sql_precision"]==0.5)].iloc[10].sql_extracted

['集尘频率', '集尘模式']

In [209]:
df_exploded[(df_exploded["recall"]==1)&(df_exploded["sql_recall"]==1)&(df_exploded["sql_precision"]==0.5)].iloc[10].pred

['集尘频率', '集尘模式', '是否支持自动集尘']

In [37]:
# 定义反查字典

In [597]:
all_dict = {}
for i in range(df_exploded.shape[0]):
    replace = df_exploded['augment'].iloc[i]["replace"]
    primaries = [j['primary_value'] for j in df_exploded['gen'].iloc[i]['prompt']]
    for key in replace:
        if replace[key] in primaries:
            print(i)
            continue
        if key in all_dict:
            all_dict[key].add(replace[key])
        else:
            all_dict[key] = set([replace[key]])

954


In [598]:
for key in all_dict:
    all_dict[key] = list(all_dict[key])
    if key not in all_dict[key]:
        all_dict[key] = all_dict[key] + [key]

In [599]:
def cherry_pick(key, value):
    if key not in all_dict:
        print(key, 'oops')
        return
    u_list = all_dict[key]
    for i in range(len(u_list)-1, -1, -1):
        if u_list[i] == value:
            u_list.pop(i)

In [600]:
cherry_pick_list = []
alias_inv_dict = {}
for k, v in all_dict.items():
    for u in v:
        if u in alias_inv_dict:
            cherry_pick_list.append((k, u, alias_inv_dict[u]))
        alias_inv_dict[u] = k

In [601]:
for i in range(len(cherry_pick_list)):
    cherry_pick(cherry_pick_list[i][0], cherry_pick_list[i][1])
    cherry_pick(cherry_pick_list[i][2], cherry_pick_list[i][1])

In [602]:
cherry_pick_list = []
alias_inv_dict = {}
for k, v in all_dict.items():
    for u in v:
        if u in alias_inv_dict:
            cherry_pick_list.append((k, u, alias_inv_dict[u]))
        alias_inv_dict[u] = k

In [603]:
cherry_pick_list

[]

In [604]:
len(alias_inv_dict)

1796